In [1]:
import os
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
arcpy.env.overwriteOutput = True

gdb_path = r'Q:\dss_workarea\mlabiadh\workspace\20240925_human_disturbance\data\data_outputs.gdb'
arcpy.env.workspace = gdb_path

In [ ]:
# create a list of watershed areas
fc= 'watershed_groups_erased'

fields = ["WATERSHED_GROUP_CODE", "WATERSHED_GROUP_NAME", "SHAPE@AREA"]

data_wshd = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        print(f'add row {rowcount} to list')
        watershed_code= row[0]
        watershed_name = row[1]
        watershed_area = row[2] / 10000 #in hectare
        data_wshd.append([watershed_code, watershed_name, watershed_area])

        rowcount += 1

In [ ]:
# create a list of watershed/rank/area values
fc= 'BC_CEF_Human_Disturb_2023_province_withRank0_withWatersheds'

fields = ["WATERSHED_GROUP_CODE","WATERSHED_GROUP_NAME", "CEF_DISTURB_GROUP" ,"CEF_DISTURB_GROUP_RANK", "SHAPE@AREA"]

data_hd = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        print(f'add row {rowcount} to list')
        watershed_code = row[0]
        watershed_name = row[1]
        hd_group = row[2]
        hd_rank = row[3]
        hd_area = row[4] / 10000 #in hectare
        data_hd.append([watershed_code, watershed_name, hd_group, hd_rank, hd_area])

        rowcount += 1

In [ ]:
# create a list of road lengths by watershed
fc= 'integrated_roads_2024_withWatersheds'

fields = ["WATERSHED_GROUP_CODE", "WATERSHED_GROUP_NAME", "SHAPE@LENGTH"]

data_rds = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        print(f'add row {rowcount} to list')
        watershed_code = row[0]
        watershed_name = row[1]
        length = row[2] / 1000 #in kilometers
        data_rds.append([watershed_code, watershed_name, length])

        rowcount += 1

In [6]:
#read the Road lengths in a dataframe
df_rds = pd.DataFrame(data_rds, columns=["Watershed_Code","Watershed_Name", "Road_Length_km"])
df_rds.head()

,Watershed_Code,Watershed_Name,Road_Length_km
0,HAYR,Hay River,0.058712
1,HAYR,Hay River,0.121709
2,HAYR,Hay River,1.722421
3,HAYR,Hay River,0.099884
4,HAYR,Hay River,0.372298


In [7]:
#read the watershed areas in a dataframe
df_wsd = pd.DataFrame(data_wshd, columns=["Watershed_Code","Watershed_Name", "Watershed_Area_ha"])
df_wsd.head()

,Watershed_Code,Watershed_Name,Watershed_Area_ha
0,UPET,Upper Petitot River,197010.681964
1,LFRT,Lower Fort Nelson River,485977.705103
2,SAHD,Sahdoanah Creek,230282.649936
3,SAHT,Sahtaneh River,406366.168111
4,HAYR,Hay River,152593.871289


In [8]:
#read the HD areas in a dataframe
df_hd = pd.DataFrame(data_hd, columns=["Watershed_Code", "Watershed_Name", "Disturbance_Group", "Disturbance_Rank", "Area_ha"])
df_hd.head()

,Watershed_Code,Watershed_Name,Disturbance_Group,Disturbance_Rank,Area_ha
0,HAYR,Hay River,OGC_Geophysical,8,0.363472
1,HAYR,Hay River,OGC_Infrastructure,3,0.288565
2,HAYR,Hay River,OGC_Geophysical,8,8.806994
3,HAYR,Hay River,OGC_Geophysical,8,2.048153
4,HAYR,Hay River,OGC_Geophysical,8,0.284612


In [9]:
#calculate total disturbance area by watershed
df_hd_tot= df_hd.groupby(["Watershed_Code", "Watershed_Name"])["Area_ha"].sum().reset_index()
df_hd_tot.rename(columns={"Area_ha": "Total_Rank_Area"}, inplace=True)
df_hd_tot['Total_Rank_Area']= round(df_hd_tot['Total_Rank_Area'],2)

#add watershed areas (for percentage calulcation)
df_hd_tot = pd.merge(
    df_hd_tot, 
    df_wsd[["Watershed_Code", "Watershed_Name", "Watershed_Area_ha"]], 
    how= 'left',
    on=["Watershed_Code", "Watershed_Name"]
)

#calculate total percent
df_hd_tot['Total_Rank_Percent'] = round((df_hd_tot["Total_Rank_Area"] / df_hd_tot["Watershed_Area_ha"]) * 100 ,5)

df_hd_tot.head()

,Watershed_Code,Watershed_Name,Total_Rank_Area,Watershed_Area_ha,Total_Rank_Percent
0,ADMS,Adams River,106085.24,312934.046482,33.90019
1,ALBN,Alberni Inlet,132057.21,368102.946501,35.87508
2,ATLL,Atlin Lake,3842.60,407888.372603,0.94207
3,ATNA,Atnarko River,10070.89,243367.716520,4.13814
4,BABL,Babine Lake,194536.00,595295.377400,32.67890


In [10]:
# caclulate Watershed Area by rank
df_hd_sum = df_hd.groupby(["Watershed_Code", "Watershed_Name","Disturbance_Group" ,"Disturbance_Rank"])["Area_ha"].sum().reset_index()
df_hd_sum.rename(columns={"Area_ha": "Rank_Area_ha"}, inplace=True)

# join the two dfs
df_hd_sum_wsd = pd.merge(
    df_wsd, 
    df_hd_sum, 
    how= 'left',
    on=["Watershed_Code", "Watershed_Name"]
)

# caluclate percentages
df_hd_sum_wsd["Percent"] = round((df_hd_sum_wsd["Rank_Area_ha"] / df_hd_sum_wsd["Watershed_Area_ha"]) * 100 ,5)

#round numbers for readability
df_hd_sum_wsd["Rank_Area_ha"] = round(df_hd_sum_wsd["Rank_Area_ha"], 2)
df_hd_sum_wsd["Watershed_Area_ha"] = round(df_hd_sum_wsd["Watershed_Area_ha"], 2)

#sort rows by watershed and rank
df_hd_sum_wsd.sort_values(by=['Watershed_Name', 'Disturbance_Rank'], inplace= True)

df_hd_sum_wsd.head()

,Watershed_Code,Watershed_Name,Watershed_Area_ha,Disturbance_Group,Disturbance_Rank,Rank_Area_ha,Percent
718,ADMS,Adams River,312934.05,Roads,0.0,12547.16,4.00952
715,ADMS,Adams River,312934.05,Mining_and_Extraction,1.0,41.12,0.01314
716,ADMS,Adams River,312934.05,Power,4.0,66.36,0.02121
717,ADMS,Adams River,312934.05,ROW,5.0,78.53,0.02509
719,ADMS,Adams River,312934.05,Urban,6.0,129.06,0.04124


In [11]:
# caclulate Road density

#calculate watershed area in sq km
df_wsd['Watershed_Area_sqkm'] = df_wsd['Watershed_Area_ha'] / 100 

#summarize road lengths by watershed
df_rds_sum = df_rds.groupby(["Watershed_Code", "Watershed_Name"])["Road_Length_km"].sum().reset_index()

#add watershed areas
df_rds_sum_dn = pd.merge(
    df_rds_sum, 
    df_wsd[['Watershed_Code','Watershed_Name','Watershed_Area_sqkm']], 
    how= 'left',
    on= ["Watershed_Code", "Watershed_Name"]
)

#caluclate road density
df_rds_sum_dn['Road_Density'] = round(df_rds_sum_dn['Road_Length_km'] / df_rds_sum_dn['Watershed_Area_sqkm'],5)

df_rds_sum_dn.head()

,Watershed_Code,Watershed_Name,Road_Length_km,Watershed_Area_sqkm,Road_Density
0,ADMS,Adams River,6598.352563,3129.340465,2.10854
1,ALBN,Alberni Inlet,10021.348826,3681.029465,2.72243
2,ATLL,Atlin Lake,1070.769479,4078.883726,0.26252
3,ATNA,Atnarko River,947.312683,2433.677165,0.38925
4,BABL,Babine Lake,10327.918983,5952.953774,1.73492


In [12]:
# add Road density to main df
df_hd_rd = pd.merge(
    df_hd_sum_wsd[['Watershed_Code', 'Watershed_Name','Watershed_Area_ha','Disturbance_Rank', 'Rank_Area_ha', 'Percent']],
    df_rds_sum_dn[['Watershed_Code','Watershed_Name', 'Road_Length_km','Road_Density']],
    how= 'left',
    on= ["Watershed_Code", "Watershed_Name"]
)

df_hd_rd.head()

,Watershed_Code,Watershed_Name,Watershed_Area_ha,Disturbance_Rank,Rank_Area_ha,Percent,Road_Length_km,Road_Density
0,ADMS,Adams River,312934.05,0.0,12547.16,4.00952,6598.352563,2.10854
1,ADMS,Adams River,312934.05,1.0,41.12,0.01314,6598.352563,2.10854
2,ADMS,Adams River,312934.05,4.0,66.36,0.02121,6598.352563,2.10854
3,ADMS,Adams River,312934.05,5.0,78.53,0.02509,6598.352563,2.10854
4,ADMS,Adams River,312934.05,6.0,129.06,0.04124,6598.352563,2.10854


In [13]:
# Force Disturbance_Rank column to integer
df_hd_rd['Disturbance_Rank'] = df_hd_rd['Disturbance_Rank'].astype('Int64')

# Rename the Area column
df_hd_rd.rename(columns={"Rank_Area_ha": "Area"}, inplace=True)
df_hd_rd.rename(columns={"Watershed_Area_ha": "WG_AREA_HA"}, inplace=True)

# Pivot the table
df_hd_rd_pv = df_hd_rd.pivot_table(
    index=["Watershed_Code", "Watershed_Name"],
    columns='Disturbance_Rank',
    values=['Area', 'Percent'],
    fill_value=0
)

# Rename columns
df_hd_rd_pv.columns = [f'Rank{int(rank)}_{col}' for col, rank in df_hd_rd_pv.columns]
df_hd_rd_pv.reset_index(inplace=True)


# Add the Watershed_Code and Road_Density columns to the pivoted table
df_hd_rd_pv = df_hd_rd[['Watershed_Code', 'Watershed_Name','WG_AREA_HA','Road_Length_km','Road_Density']].drop_duplicates().merge(
    df_hd_rd_pv, on= ["Watershed_Code", "Watershed_Name"]
)

# Fill NA values in Road_Density with 0
df_hd_rd_pv = df_hd_rd_pv.fillna(0)

# Sort columns
sorted_columns = ['Watershed_Code', 'Watershed_Name','WG_AREA_HA' ,'Road_Length_km','Road_Density'] + sorted(
    [col for col in df_hd_rd_pv.columns if col not in ['Watershed_Code', 'Watershed_Name','WG_AREA_HA' , 'Road_Length_km','Road_Density']],
    key=lambda x: (int(x.split('_')[0][4:]), x.split('_')[1])
)

df_hd_rd_pv = df_hd_rd_pv[sorted_columns]


#add the total rank area
df_hd_rd_pv = pd.merge(
    df_hd_rd_pv,
    df_hd_tot[["Watershed_Code", "Watershed_Name", 'Total_Rank_Area', 'Total_Rank_Percent' ]],
    how= 'left',
    on= ["Watershed_Code", "Watershed_Name"]
)

df_hd_rd_pv.head()


,Watershed_Code,Watershed_Name,WG_AREA_HA,Road_Length_km,Road_Density,Rank0_Area,Rank0_Percent,Rank1_Area,Rank1_Percent,Rank2_Area,...,Rank7_Area,Rank7_Percent,Rank8_Area,Rank8_Percent,Rank9_Area,Rank9_Percent,Rank10_Area,Rank10_Percent,Total_Rank_Area,Total_Rank_Percent
0,ADMS,Adams River,312934.05,6598.352563,2.10854,12547.16,4.00952,41.12,0.01314,0.00,...,0.00,0.00000,0.0,0.0,91642.99,29.28508,1580.02,0.50491,106085.24,33.90019
1,ALBN,Alberni Inlet,368102.95,10021.348826,2.72243,18179.19,4.93862,354.32,0.09626,47.11,...,247.08,0.06712,0.0,0.0,106862.18,29.03052,1086.11,0.29506,132057.21,35.87508
2,ATLL,Atlin Lake,407888.37,1070.769479,0.26252,2263.63,0.55496,1136.46,0.27862,0.00,...,0.00,0.00000,0.0,0.0,228.43,0.05600,0.00,0.00000,3842.60,0.94207
3,ATNA,Atnarko River,243367.72,947.312683,0.38925,1820.29,0.74796,0.00,0.00000,0.00,...,79.59,0.03270,0.0,0.0,7924.77,3.25630,94.91,0.03900,10070.89,4.13814
4,BABL,Babine Lake,595295.38,10327.918983,1.73492,17933.22,3.01249,1303.34,0.21894,0.00,...,0.00,0.00000,0.0,0.0,174109.87,29.24764,101.95,0.01713,194536.00,32.67890


In [14]:
# join the table to the watersheds featureclass

# load the watershed groups featureclass as esri geodataframe
wsh_sedf= pd.DataFrame.spatial.from_featureclass(
    f"{gdb_path}/watershed_groups"
)

In [15]:
#filter unecessary columns
wsh_sedf = wsh_sedf[['WATERSHED_GROUP_ID', 'WATERSHED_GROUP_CODE',
                     'WATERSHED_GROUP_NAME', 'AREA_HA', 'SHAPE']]



#join stats from the pivoted table
df_sp= pd.merge(
    wsh_sedf,
    df_hd_rd_pv,
    how= 'left',
    left_on=['WATERSHED_GROUP_CODE', 'WATERSHED_GROUP_NAME'],
    right_on=['Watershed_Code', 'Watershed_Name']
 )  

#fill nulls
df_sp.fillna({col: 0 for col in df_sp.columns if col != 'SHAPE'}, inplace=True)

 #remove the extra watershed name and code columns 
df_sp.drop(['Watershed_Code', 'Watershed_Name'], axis=1, inplace= True)

df_sp.head()

,WATERSHED_GROUP_ID,WATERSHED_GROUP_CODE,WATERSHED_GROUP_NAME,AREA_HA,SHAPE,WG_AREA_HA,Road_Length_km,Road_Density,Rank0_Area,Rank0_Percent,...,Rank7_Area,Rank7_Percent,Rank8_Area,Rank8_Percent,Rank9_Area,Rank9_Percent,Rank10_Area,Rank10_Percent,Total_Rank_Area,Total_Rank_Percent
0,170,PORI,Porcher Island,76830.421705,"{""rings"": [[[714480.6779999994, 1016992.074999...",76697.54,226.461375,0.29527,394.26,0.51405,...,0.00,0.00000,0.00,0.00000,3448.59,4.49635,0.00,0.00000,3956.93,5.15914
1,236,UPET,Upper Petitot River,203221.20955,"{""rings"": [[[1335718.1199999992, 1627927.03900...",197010.68,1332.125336,0.67617,2711.95,1.37655,...,0.00,0.00000,3274.87,1.66228,0.00,0.00000,0.00,0.00000,6468.99,3.28357
2,99,LCHL,Lower Chilako River,313884.871279,"{""rings"": [[[1203208.9230000004, 1009122.62800...",308121.02,8649.850881,2.80729,17162.35,5.57000,...,271.45,0.08810,0.00,0.00000,129451.18,42.01310,14613.92,4.74291,167959.98,54.51104
3,62,GUIC,Guichon Creek,122383.33189,"{""rings"": [[[1394363.1469999999, 624560.342000...",121687.08,3795.473868,3.11904,7181.96,5.90199,...,206.92,0.17004,0.00,0.00000,44391.98,36.48044,2844.47,2.33753,62300.72,51.19748
4,131,MESC,Mess Creek,233030.040893,"{""rings"": [[[683708.3680000007, 1393889.173000...",204709.51,98.595369,0.04816,179.01,0.08745,...,0.00,0.00000,0.00,0.00000,55.38,0.02705,0.00,0.00000,249.84,0.12205


In [ ]:
#export the watershed dataset with stats to gdb
df_sp.spatial.to_featureclass(location=f"{gdb_path}/watershed_groups_stats")